### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [62]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data_df = pd.read_csv(file_to_load)
purchase_data_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Player Count

* Display the total number of players


In [27]:
#Calculate total number of players
total_players = len(purchase_data_df["SN"].unique())

#Create and Display Dataframe
total_players_data = [[total_players]]
summary_df = pd.DataFrame(total_players_data, columns = ["Total Players"])
summary_df.head()

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [28]:
#Calculate summary
unique_items = len(purchase_data_df["Item Name"].unique())
average_price = purchase_data_df["Price"].mean()
total_purchases = purchase_data_df["SN"].count()
total_revenue = purchase_data_df["Price"].sum()

#Create DataFrame
summary_data = [[unique_items, average_price, total_purchases, total_revenue]]
summary_df = pd.DataFrame(summary_data, columns = ["Number of Unique Items", "Average Price", "Number of Purchases", "Total Revenue"])

#Format DataFrame to include currency symbpl
summary_df["Average Price"] = summary_df["Average Price"].astype(float).map("${:,.2f}".format)
summary_df["Total Revenue"] = summary_df["Total Revenue"].astype(float).map("${:,.2f}".format)
summary_df.head()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [29]:
# Get unique DataFrame of users as a single player can buy multiple items
unique_users_df= purchase_data_df.groupby(["SN", "Gender"], as_index=False).first()

# Create DataFrame off of the unique users lists, as value counts - rename as appropriate
gender_df = pd.DataFrame(unique_users_df["Gender"].value_counts())
gender_df = gender_df.rename(
    columns={"Gender": "Total Count"})

# Calculate Percentage and add to the datafame, reformat percentage to 
percentage_of_players = gender_df["Total Count"]/total_players*100
gender_df["Percentage of Players"] = percentage_of_players
gender_df["Percentage of Players"] = gender_df["Percentage of Players"].astype(float).map("{0:.2f}%".format)

gender_df = gender_df.reset_index()
gender_df = gender_df.rename(columns={"index": "Gender"})

gender_df.head()

,Gender,Total Count,Percentage of Players
0,Male,484,84.03%
1,Female,81,14.06%
2,Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [30]:
#Calculate Purchase Count
purchase_by_gender_df = pd.DataFrame(purchase_data_df["Gender"].value_counts())
purchase_by_gender_df = purchase_by_gender_df.rename(
    columns={"Gender": "Purchase Count"})

purchase_by_gender_df = purchase_by_gender_df.reset_index()
purchase_by_gender_df = purchase_by_gender_df.rename(columns={"index": "Gender"})

#Calculate Average Purchase Price per Gender
average_price_by_gender_df = pd.DataFrame(purchase_data_df.groupby("Gender")["Price"].mean())
average_price_by_gender_df = average_price_by_gender_df.rename(columns={"Price": "Average Purchase Price"})
average_price_by_gender_df["Average Purchase Price"] = average_price_by_gender_df["Average Purchase Price"].astype(float).map("${:,.2f}".format)
purchase_by_gender_df = purchase_by_gender_df.merge(average_price_by_gender_df, on="Gender")

#Calculate Total Purchase Value per Gender
total_purchase_price_by_gender_df = pd.DataFrame(purchase_data_df.groupby("Gender")["Price"].sum())
total_purchase_price_by_gender_df = total_purchase_price_by_gender_df.rename(columns={"Price": "Total Purchase Value"})
# total_purchase_price_by_gender_df["Total Purchase Value"] = total_purchase_price_by_gender_df["Total Purchase Value"].astype(float).map("${:,.2f}".format)
purchase_by_gender_df = purchase_by_gender_df.merge(total_purchase_price_by_gender_df, on="Gender")

#Calculate Average Total Purchase per Person
#Temporarily merge gender_df, which holds unique counts for gender - we'll use the unique counts to find average
purchase_by_gender_df = purchase_by_gender_df.merge(gender_df, on="Gender")
purchase_by_gender_df["Avg Total Purchase per Person"] = purchase_by_gender_df["Total Purchase Value"]/purchase_by_gender_df["Total Count"]

#Format the currencies
purchase_by_gender_df["Total Purchase Value"] = purchase_by_gender_df["Total Purchase Value"].astype(float).map("${:,.2f}".format)
purchase_by_gender_df["Avg Total Purchase per Person"] = purchase_by_gender_df["Avg Total Purchase per Person"].astype(float).map("${:,.2f}".format)

#Drop unnecessary columns as we've calculated and inserted average values
purchase_by_gender_df = purchase_by_gender_df.drop(columns=['Total Count', 'Percentage of Players'])

purchase_by_gender_df.head()

,Gender,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0,Male,652,$3.02,"$1,967.64",$4.07
1,Female,113,$3.20,$361.94,$4.47
2,Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [77]:
#Setup Bins and apply cut - new columne Age Group added 
bins = [0, 9, 14, 19, 24, 29, 34, 39, 50]
age_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
purchase_data_df["Age Group"] = pd.cut(purchase_data_df["Age"], bins, labels=age_labels)

#Create final Age Demo summary table. 
age_summary_df = pd.DataFrame(purchase_data_df["Age Group"].value_counts())
age_summary_df = age_summary_df.reset_index()       
age_summary_df = age_summary_df.rename(columns={"index": "Age Ranges"})
age_summary_df = age_summary_df.sort_values("Age Ranges")

#Calculate Average Purchase Price
average_price_by_age_df = pd.DataFrame(purchase_data_df.groupby("Age Group")["Price"].mean())
average_price_by_age_df = average_price_by_age_df.rename(columns={"Price": "Average Purchase Price"})
average_price_by_age_df["Average Purchase Price"] = average_price_by_age_df["Average Purchase Price"].astype(float).map("${:,.2f}".format)
average_price_by_age_df = average_price_by_age_df.reset_index()       
average_price_by_age_df = average_price_by_age_df.rename(columns={"Age Group": "Age Ranges"})

age_summary_df = age_summary_df.merge(average_price_by_age_df, on="Age Ranges")

#Calculate Total Purchase Value per Gender
total_purchase_price_by_age_df = pd.DataFrame(purchase_data_df.groupby("Age Group")["Price"].sum())
total_purchase_price_by_age_df = total_purchase_price_by_age_df.rename(columns={"Price": "Total Purchase Value"})
total_purchase_price_by_age_df = total_purchase_price_by_age_df.reset_index() 
total_purchase_price_by_age_df = total_purchase_price_by_age_df.rename(columns={"Age Group": "Age Ranges"})
total_purchase_price_by_age_df

age_summary_df = age_summary_df.merge(total_purchase_price_by_age_df, on="Age Ranges")


#Calculate Avg Total Purchase per Person
#Drop duplicate user rows to get approprate total count for each group. Unique user group number will be used
#to calculate average
age_summary_unique_df = pd.DataFrame(purchase_data_df.drop_duplicates(subset=["SN"]))
age_summary_uniques_df = pd.DataFrame(age_summary_unique_df["Age Group"].value_counts())
age_summary_uniques_df = age_summary_uniques_df.reset_index()       
age_summary_uniques_df = age_summary_uniques_df.rename(columns={"index": "Age Ranges"})
age_summary_uniques_df = age_summary_uniques_df.sort_values("Age Ranges")
age_summary_df = age_summary_df.merge(age_summary_uniques_df, on="Age Ranges")
age_summary_df = age_summary_df.rename(columns={"Age Group_y": "Total Count"})

#Calucalate Average
age_summary_df["Avg Total Purchase per Person"] = age_summary_df["Total Purchase Value"]/age_summary_df["Total Count"]

#Format the currencies
age_summary_df["Total Purchase Value"] = age_summary_df["Total Purchase Value"].astype(float).map("${:,.2f}".format)
age_summary_df["Avg Total Purchase per Person"] = age_summary_df["Avg Total Purchase per Person"].astype(float).map("${:,.2f}".format)

#Drop unnecessary columns as we've calculated and inserted average values
age_summary_df = age_summary_df.drop(columns=['Total Count'])
age_summary_df = age_summary_df.rename(columns={"Age Group_x": "Purchase Count"})

age_summary_df

,Age Ranges,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
0,<10,23,$3.35,$77.13,$4.54
1,10-14,28,$2.96,$82.78,$3.76
2,15-19,136,$3.04,$412.89,$3.86
3,20-24,365,$3.05,"$1,114.06",$4.32
4,25-29,101,$2.90,$293.00,$3.81
5,30-34,73,$2.93,$214.00,$4.12
6,35-39,41,$3.60,$147.67,$4.76
7,40+,13,$2.94,$38.24,$3.19


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, average item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

